# Welcome to Justuse!

## Installation
Before we start, let's get the latest version

In [1]:
%%bash
python -m pip install justuse

In [2]:
import use

In [3]:
use.__version__

'0.6.5'

## Basic Usage

Let's start with a simple case

In [4]:
use("math").cos(23)

-0.5328330203333975

Installing something could be inconvenient or unnecessary if something else is available - or we want to include some minimal functionality in our program and only fetch additional dependencies only under certain conditions.

The common approach would be something like

In [5]:
try:
    import some_big_package
except ImportError:
    some_big_package = None
if some_big_package:
    ...

which is unnecessarily cumbersome - couldn't we simply have a default like in so many other functions that is returned instead of raising an exception? Of course we can!

Here's a metaphor from *The Matrix*:
[![Matrix - Skill Upload](https://img.youtube.com/vi/w_8NsPQBdV0/0.jpg)](https://www.youtube.com/watch?v=w_8NsPQBdV0)

Imagine you want to streamline the user experience by distributing a very minimal, "free" but fully functional software to your end users which installs within seconds. Now, whenever the user wants to use a premium feature (or simply a feature that isn't generally required by the majority of users, therefor not included in the basic installation) the program could use() the packages and modules needed to realise the feature to download and install in the background while the user can still use other stuff, then trigger a callback when use() is done loading. The experience would be similar to playing an open world game which seamlessly downloads and loads new areas in the background on demand, without hiccup or loading screens. Or like Neo and Trinity - just get the skills to pilot a helicopter when you need them, right there on the spot. 

In [6]:
pkg = use("some_big_package", default=None)
if not pkg:
    print("I'm going to learn Ju-Jutsu?")

I'm going to learn Ju-Jutsu?


Even more concise (python >3.9):

In [7]:
if (pkg := use("pytest", default=None)):
    print("I know Kung Fu!")

I know Kung Fu!


One of the most practical use()s is making sure we imported the expected version of a certain package. This is especially important in research papers, notebooks and other publications because those often don't come with a `requirements.txt` and there is no way to make sure you are actually running the published code with the same versions as the author.
Also, if you pip-install something, it can happen that it upgrades a dependency, accidentally breaking code that requires the old version - with pip you *can't have more than one version installed*. With justuse any number of versions can be installed in parallel, without interfering with anything that was installed globally via pip, conda etc.

In [8]:
np = use("numpy", version="2022")

/home/thorsten/anaconda3/lib/python3.8/site-packages/use/pimp.py:81: VersionWarning: numpy expected to be version 2022, but got 1.19.5 instead
  warn(Message.version_warning(name, version, result_version), category=VersionWarning)


In [9]:
np.__version__

'1.19.5'

Here you see that even though we got a warning about the wrong version, we still get the requested package, just giving you a heads up about a possibly problematic situation without standing in the way.

Let's try another one!

In [10]:
pg = use("pygame")

ImportError: No pkg installed named pygame and auto-installation not requested. Aborting.

Well, bummer! We want to play with pygame, let's have pygame!

In [11]:
pg = use("pygame", modes=use.auto_install)

RuntimeWarning: Please specify version and hash for auto-installation of 'pygame'.
A webbrowser will open to the Snyk Advisor to check whether the package is vulnerable.
If you want to auto-install the latest version:
use("pygame", version="2.1.0", hashes={'J娼䥺瑹鍖谫癦䯅恢搸烃诺嬝矫暠鸤嘀邦'}, modes=use.auto_install)

Now we're getting somewhere! Hmm.. it says "To get some valuable insight on the health of this package, please check out https://snyk.io/advisor/python/pygame - see for yourself!

You wonder what those chinese looking characters are? Well. Normally, hexdigests look something like `d6c1c1c53a988b3daf44c1865d40f86de48665639bfbd5eea1317eb083638a3a` which is way too verbose on a normal line of code and since you shouldn't manually type those anyway but only copy&paste, we thought what the heck - let's use the japanese, ascii, chinese and korean alphabets (thus JACK) to encode those hashes as compact as we can.

Have a look at the last line of the Error message - let's try to copy&paste it..

In [12]:
use("pygame", version="2.1.0", hashes={'J娼䥺瑹鍖谫癦䯅恢搸烃诺嬝矫暠鸤嘀邦'}, modes=use.auto_install)

pygame 2.1.0 (SDL 2.0.16, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


<module 'pygame' from '/home/thorsten/.justuse-python/venv/pygame/2.1.0/lib/python3.8/site-packages/pygame/__init__.py'>

Wow, did we just download, install and load the spam package without leaving our own sweet code?? Yes, we did!

Furthermore, the package we installed is version and hash-pinned so we really only get what we asked for and nothing else.

There's a small problem though. Those hashes refer to very specific files and some of those packages may be written in C or even Fortran (like numpy) that are compiled for specific platforms.
If you happily develop code on Linux that uses something platform-specific (like numpy!) it will all work without problems - until you try to run your code on another platform. In this case, you need to specify all hashes for all platforms you want to run your code on.

Version- and hash-pinning is the most secure way to install a package. It will ensure that your code will always run as you expect it, but there's a drawback: there is no immediate and automatic way to update code without involving the user (yet). On one side, you won't ever accidentally break your stuff by updating something else, but you also won't benefit from automatic security patches. To fix this shortcoming, it might be feasible to build IDE-plugins that check and update these pins in the code or check some database for security patches every time an auto-installed package is imported - please contact us if you have ideas or better yet, code ;-)

## Use() modules from anywhere!
If you `import` some package or module, you're limited to the stuff you have in your current directory or below (but only if there is a `__init__.py` or if it's an implicit namespace package) and the things in your sys.path, which can be manipulated freely, making it very complicated to handle. Let's suppose we're in our test directory.

In [13]:
%cd ~/Desktop/sf_Dropbox/code/justuse/tests

/media/sf_Dropbox/code/justuse/tests


the code we want to run is in justuse/docs and there is no `__init__.py` in between, so to get to run the code, we could put the src directory in sys.path - or we could use() a module directly!

In [14]:
mod = use(use.Path("../docs/demo.py"))

In [15]:
mod.foo()

Hello justuse-user!


Loading single modules doesn't sound like much, but especially while experimenting on jupyter, this can be used very effectively in conjunction with the reloading mode:

In [16]:
mod = use(use.Path("../docs/demo.py"), modes=use.reloading)

Now this module is loaded fresh whenever you modify and save the file, replacing the implementation behind the scene. This will work without any problems as long as you put functions in that module and if you access those functions via attribute-access (`mod.func()` **not** `func = mod.func; func()`).

If you can load modules from disk, why couldn't you load them from the web? Let's say you found an interesting github repo like https://github.com/amogorkon/justuse. Chances are, there's also a package on pypi you can pip-install, but maybe there's not. Maybe you're only interested in a single module from that repo/package, so you don't even want to install anything. Then you could download it from github, move the file manually into your folder and import it - sounds like a lot of trouble for a single module!
There has to be a better way! And there is - you can just use() web resources:

In [17]:
mod = use(use.URL("https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py"))

/home/thorsten/anaconda3/lib/python3.8/site-packages/use/main.py:416: NoValidationWarning: Attempting to import from the interwebs with no validation whatsoever!
To safely reproduce:
use(use.URL('https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py'), hash_algo=use.Hash.sha256, hash_value='59eff31bb220ce933ccc083b9306020ec25d19d43de30e5ad4341b355d4b48bf')
  warn(Message.no_validation(url, hash_algo, this_hash), NoValidationWarning)


copy&paste that line from the exception to get that sweet hash..

In [18]:
mod = use(use.URL('https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py'), hash_algo=use.Hash.sha256, hash_value='59eff31bb220ce933ccc083b9306020ec25d19d43de30e5ad4341b355d4b48bf')

In [19]:
mod.foo()

Hello justuse-user!


Since the content of this file is now hash-pinned, it doesn't matter whether or not someone hacks github and changes the code - justuse will instantly notice before executing any code. You can even execute code directly from pastebin or any other untrusted, public platform - as long as you have the proper hash, you're safe.

## A word on circular imports
Everyone stumbles over a circular import once they try to build slightly more complex projects and it can get very ugly and overly frustrating to deal with those.

Let's suppose we have two modules A and B:

In [11]:
%cd ../docs
%ll

/media/sf_Dropbox/code/justuse/docs
insgesamt 47
-rwxrwx--- 1 root    43 Nov 23 01:13 demo.py*
-rwxrwx--- 1 root    97 Nov 26 01:35 module_a.py*
-rwxrwx--- 1 root    47 Nov 26 01:36 module_b.py*
-rwxrwx--- 1 root    62 Nov 26 01:12 module_circular_a.py*
-rwxrwx--- 1 root    68 Nov 26 01:12 module_circular_b.py*
drwxrwx--- 1 root  4096 Nov 26 01:12 __pycache__/
-rwxrwx--- 1 root 80661 Nov 26 01:38 Showcase.ipynb*


In [21]:
%less module_circular_a.py

print("Hello from A!")

foo = 23

import module_circular_b


In [22]:
%less module_circular_b.py

print("Hello from B!")


In [1]:
import module_circular_a

Hello from A!


ImportError: cannot import name 'foo' from partially initialized module 'module_circular_a' (most likely due to a circular import) (/media/sf_Dropbox/code/justuse/docs/module_circular_a.py)

In [13]:
%less module_a.py

import use

print("Hello from A!")

use(use.Path("module_b.py"), initial_globals={"foo": 23})


In [14]:
%less module_b.py

foo: int

print(f"Hello from B! foo={foo}")


In [9]:
modA = use(use.Path("module_a.py"))

Hello from A!
Hello from B! foo=23


## Aspects and decorators
If you have code like

```
def foo(x):
    return x ** 2
```
There are two ways to modify the behaviour of the code. Let's say you want to know which arguments were passed in and you want to print those arguments, so you can either add the print statement inside like
```
def foo(x):
    print(x)
    return x ** 2
```
or you can wrap the function with another function which gets called instead like

```
def decorator(x):
    print(x)
    return foo(x)
```
which you can also write as a function that takes a function as argument - a "higher order function" if you will - like
```
def decorator(func):
   def wrapper(*args, **kwargs):
       print(*args, **kwargs)
       return func(*args, **kwargs)
   return wrapper
foo = wrap(foo)
foo(x)
```
which looks a bit convoluted, so there is some syntactic sugar in python to make it look nicer:
```
@decorator
def foo(x):
    return x ** 2
```

There is nothing wrong with this approach, it's a plain and simple decorator, obvious reading the code. The only big problem is that single decorator is nice and simple but adding more quickly gets messy and complicated, not just because it gets hard to read and reason about but also because writing correct decorators is not as plain and simple as it may look like. For example, the *decorator* function above is bad because it will shadow the signature and docstring of the function it is wrapping. The second big problem is that it is quite messy to add decorators manually to code with lots of functions or methods. It can be not only a lot to write but also add visual noise, making it harder to read code (the opposite of what it was supposed to do!) while making it harder to keep DRY. The third big problem, no wait, amongst the chief problems of manually adding decorators is that it can be quite painful or impossible to deal with callables (functions, methods but also classes can be callable!) that originate in code you can't control, for example code that isn't even written in python but in C (like numpy).

Our solution: use() adds functionality to the modules you request and gives you an easy way to decorate *everything* in a single line:

In [15]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(func, args, kwargs)
        return func(*args, **kwargs)
    return wrapper

np = use("numpy")

In [16]:
np @ (use.isfunction, "", decorator)

<module 'numpy' from '/home/thorsten/anaconda3/lib/python3.8/site-packages/numpy/__init__.py'>

Let's have a look at our log file to see what just happened:

In [17]:
%%bash
tail -100 /home/thorsten/.justuse-python/usage.log

16:29:02,248 use.pimp DEBUG Applied decorator to numpy::take_along_axis [function]
16:29:02,248 use.pimp DEBUG Applied decorator to numpy::put_along_axis [function]
16:29:02,248 use.pimp DEBUG Applied decorator to numpy::broadcast_to [function]
16:29:02,248 use.pimp DEBUG Applied decorator to numpy::broadcast_arrays [function]
16:29:02,248 use.pimp DEBUG Applied decorator to numpy::diag [function]
16:29:02,248 use.pimp DEBUG Applied decorator to numpy::diagflat [function]
16:29:02,248 use.pimp DEBUG Applied decorator to numpy::eye [function]
16:29:02,248 use.pimp DEBUG Applied decorator to numpy::fliplr [function]
16:29:02,249 use.pimp DEBUG Applied decorator to numpy::flipud [function]
16:29:02,249 use.pimp DEBUG Applied decorator to numpy::tri [function]
16:29:02,249 use.pimp DEBUG Applied decorator to numpy::triu [function]
16:29:02,249 use.pimp DEBUG Applied decorator to numpy::tril [function]
16:29:02,250 use.pimp DEBUG Applied decorator to numpy::vander [function]
16:29:02,250 us

We wrapped *all* functions in the numpy package with a decorator that lets us directly observe what happens when we call something without the need for an external debugger!

In [18]:
np.array([1,2,3])

<function amax at 0x7fd13028baf0> (array(...),) {}
<function amin at 0x7fd13028bca0> (array(...),) {}


array([1, 2, 3])

If you aren't in a situation where you don't need to debug, log or control access (the classical applications of AOP) you might still want to use it in python - for type-checking/testing!
For example you can apply our favorite dynamic type checker library beartype quite easily like so:

In [20]:
from beartype import beartype

np @ (use.isfunction, "", beartype)

<module 'numpy' from '/home/thorsten/anaconda3/lib/python3.8/site-packages/numpy/__init__.py'>

Now all functions in numpy are beartype-checked! You can do the same with your own code, as long as you use() your modules.

If you want to learn more about aspect-oriented programming, check out https://en.wikipedia.org/wiki/Aspect-oriented_programming 